<h1>Para começar, importa-se as libs necessárias para os procedimentos de data cleaning juntamente com os dataframes separadamente, um por um</h1>

In [223]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from os.path import abspath
import os

# Remove o arquivo final caso ele exista no diretório dos databases
try:
    pd.read_csv(abspath('./data/dataBase.gen.csv'))
except:
    print("Não existe o arquivo dataBase.gen.csv")
else:
    os.remove(abspath('./data/dataBase.gen.csv'))

customer = pd.read_csv(abspath('./data/olist_customers_dataset.csv'))
geo_location = pd.read_csv(abspath('./data/olist_geolocation_dataset.csv'))
items = pd.read_csv(abspath('./data/olist_order_items_dataset.csv'))
payments = pd.read_csv(abspath('./data/olist_order_payments_dataset.csv'))
reviews = pd.read_csv(abspath('./data/olist_order_reviews_dataset.csv'))
orders = pd.read_csv(abspath('./data/olist_orders_dataset.csv'))
products = pd.read_csv(abspath('./data/olist_products_dataset.csv'))
sellers = pd.read_csv(abspath('./data/olist_sellers_dataset.csv'))
translation = pd.read_csv(
    abspath('./data/product_category_name_translation.csv'))


Não existe o arquivo dataBase.csv


<h2>Cria-se função para obter demais informações dos dados</h2>


In [224]:
def dfOverview(dataframe):
    """This function will return the overview of the dataframe"""

    print("O tamanho do dataframe é (linha, coluna): {}".format(dataframe.shape))
    print("**"*30)
    print("Informações: ", dataframe.info())
    print("**"*30)
    print("Número total de valores nulos: \n ", dataframe.isnull().sum())
    print("**"*30)

    return dataframe.head(3)


<h2>Por exemplo, pode-se analisar os dados da base de reviews:</h2>

In [225]:
dfOverview(geo_location)

O tamanho do dataframe é (linha, coluna): (1000163, 5)
************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB
Informações:  None
************************************************************
Número total de valores nulos: 
  geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP


<h2><strong>Observações<strong></h2>


<ul>
<li>Valores nulos se fazem presentes nos seguintes dataframes: review, orders e product;</li>

<li>Em *geo_location* temos um dataframe com 1000163 pontos o que, em relação aos outros dataframes, aparenta ser bastante grande. Isso nos leva a conclusão de que possamos ter dados duplicados nesse dataframe;</li>

<li>Precisaremos mesclar todas as tabelas de acordo com esquema a nós fornecido.</li>
</ul>

<h2>Padronizando nomes das colunas de coordenadas geográficas em 3 diferentes dataframes</h2>

In [226]:
customer.rename(
    columns={"customer_zip_code_prefix": "zip_code_prefix"}, inplace=True)
geo_location.rename(
    columns={"geolocation_zip_code_prefix": "zip_code_prefix"}, inplace=True)
sellers.rename(
    columns={"seller_zip_code_prefix": "zip_code_prefix"}, inplace=True)


<h2>Removendo valores duplicados em geo_location df</h2>

In [227]:
print("Número de linhas antes da remoção: ", geo_location.shape[0])
geo_location.drop_duplicates(
    subset="zip_code_prefix", keep="first", inplace=True)
print("Número de linhas após remoção: ",
      geo_location.shape[0])


Número de linhas antes da remoção:  1000163
Número de linhas após remoção:  19015


<h2>Realizando o merge dos dataframs (merge é uma espécie de union que se realiza em banco de dados)</h2>

In [228]:
# customer com geo_location (em left join para priorizar infos de customer)
geo_customer = pd.merge(customer, geo_location,
                        how="left", on="zip_code_prefix")

# sellers com geo_location (em left join para priorizar infos de sellers)
geo_seller = pd.merge(sellers, geo_location, how="left", on="zip_code_prefix")

# payment com order
payment_order = pd.merge(payments, orders, on="order_id")

# product com item
product_item = pd.merge(products, items, on="product_id")

# payment_order com reviews
pay_order_review = pd.merge(payment_order, reviews, on="order_id")

# pay_order_review com geo_customer
review_customer = pd.merge(pay_order_review, geo_customer, on="customer_id")

# prod_item com geo_seller
prod_item_seller = pd.merge(product_item, geo_seller, on="seller_id")

# Finalmente, criando a database geral com review_customer com prod_item_seller
dataBase = pd.merge(review_customer, prod_item_seller,
                    on="order_id", suffixes=("_customer", "_seller"))

print("O tamanho da database é: ", dataBase.shape)
print("Todas as colunas nesse db são: \n", dataBase.columns)
num_tot_rows = dataBase.shape[0]

O tamanho da database é:  (118315, 47)
Todas as colunas nesse db são: 
 Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_unique_id',
       'zip_code_prefix_customer', 'customer_city', 'customer_state',
       'geolocation_lat_customer', 'geolocation_lng_customer',
       'geolocation_city_customer', 'geolocation_state_customer', 'product_id',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'order_item_id', 'seller_id', 'ship

<h2>Não necessitaremos de algumas informações dessas colunas, como por ex: review_id, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp</h2>

In [229]:
dataBase.drop(['review_id', 'review_comment_title', 'review_comment_message',
               'review_creation_date', 'review_answer_timestamp'], axis=1, inplace=True)

dataBase.isnull().sum()


order_id                            0
payment_sequential                  0
payment_type                        0
payment_installments                0
payment_value                       0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1254
order_delivered_customer_date    2588
order_estimated_delivery_date       0
review_score                        0
customer_unique_id                  0
zip_code_prefix_customer            0
customer_city                       0
customer_state                      0
geolocation_lat_customer          317
geolocation_lng_customer          317
geolocation_city_customer         317
geolocation_state_customer        317
product_id                          0
product_category_name            1709
product_name_lenght              1709
product_description_lenght       1709
product_photos_qty               1709
product_weig

<h2>Obtendo os status de entrega do pedido</h2>

In [230]:
dataBase["order_status"].value_counts()

delivered      115728
shipped          1255
canceled          570
invoiced          376
processing        376
unavailable         7
approved            3
Name: order_status, dtype: int64

<h2>Apenas os produtos entregues nos interessa, então temos:</h2>

In [231]:
not_delivered = dataBase[dataBase["order_status"] != "delivered"]
not_delivered = not_delivered["order_status"].value_counts().sum()

print("Total de pedidos não entregues: {} | o que corresponde a {}% dos pedidos".format(
    not_delivered, (not_delivered*100/dataBase.shape[0]).round(3)))

delivered = dataBase[dataBase["order_status"]=="delivered"]["order_status"].value_counts().sum() 
print((delivered*100/dataBase.shape[0]).round(3))

Total de pedidos não entregues: 2587 | o que corresponde a 2.187% dos pedidos
97.813


In [232]:
# excluindo linhas nulas nas colunas de datas
dataBase.dropna(subset=["order_approved_at", "order_delivered_carrier_date",
                "order_delivered_customer_date"], axis=0, inplace=True)
dataBase.isnull().sum()

# excluindo linhas nulas nas colunas de geo_location mandantes
dataBase.dropna(subset=["geolocation_lng_seller",
                "geolocation_state_customer"], axis=0, inplace=True)
dataBase.isnull().sum()

# exluindo outras linhas
dataBase.dropna(subset=["product_width_cm",
                "product_photos_qty"], axis=0, inplace=True)

dataBase.isnull().sum()
dataBase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113516 entries, 0 to 118314
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113516 non-null  object 
 1   payment_sequential             113516 non-null  int64  
 2   payment_type                   113516 non-null  object 
 3   payment_installments           113516 non-null  int64  
 4   payment_value                  113516 non-null  float64
 5   customer_id                    113516 non-null  object 
 6   order_status                   113516 non-null  object 
 7   order_purchase_timestamp       113516 non-null  object 
 8   order_approved_at              113516 non-null  object 
 9   order_delivered_carrier_date   113516 non-null  object 
 10  order_delivered_customer_date  113516 non-null  object 
 11  order_estimated_delivery_date  113516 non-null  object 
 12  review_score                  

<h2>A quantidade total de dados retirado foi:</h2>

In [233]:
print(round((num_tot_rows - dataBase.shape[0])*100/num_tot_rows, 3), "%")


4.056 %


<h2>O total de <strong>4,056%</strong> dos dados é relativamente pouco, já que ainda nos resta cerca de 96% dos dados sem valores nulos</h2>

<h2>Removendo as linhas duplicadas:</h2>

In [234]:
dataBase.drop_duplicates(subset=["order_id"], keep="first", inplace=True)
dataBase[dataBase.duplicated()==True]

,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,shipping_limit_date,price,freight_value,zip_code_prefix_seller,seller_city,seller_state,geolocation_lat_seller,geolocation_lng_seller,geolocation_city_seller,geolocation_state_seller


<h2>Nas infos de seller e customer, temos os nomes das cidades sem caracteres especiais, além de algumas cidade com seus estados correspondetes errados. Em geolocations essas diferenças são corrigidas, logo, aproveitaremos essas informações de geolocation e excluiremos as colunas de cidade/estado de seller e customer</h2>

In [235]:
dataBase[dataBase["geolocation_state_seller"] !=
         dataBase["seller_state"]]  # não está igual

dataBase.drop(["seller_state", "seller_city", "customer_city",
               "customer_state"], axis=1, inplace=True)

dataBase.rename(columns={"geolocation_state_seller": "seller_state", "geolocation_city_seller": "seller_city",
                         "geolocation_lng_seller": "lng_seller", "geolocation_lat_seller": "lat_seller",
                         "geolocation_lng_customer": "lng_customer", "geolocation_lat_customer": "lat_customer",
                         "geolocation_city_customer": "customer_city", "geolocation_state_customer": "customer_state"}, inplace=True)


<h2>Temos 2 linhas com parcela igual a 0, logo, devemos eliminá-la, pois não há condição de termos um produto com número de parcelas igual a 0.</h2>

In [236]:
print(dataBase[dataBase["payment_installments"] == 0].values.shape[0])

idx = dataBase[dataBase["payment_installments"] == 0].index.values

dataBase.drop(dataBase.index[idx], axis=0, inplace=True)

2


<h2>Por fim, salvamos os dados em um novo arquivo .csv</h2>

In [237]:
dataBase.to_csv(abspath("data/dataBase.gen.csv"), index=False)
